In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.seqsignet_attention_encoder_functions import (
    seqsignet_attention_encoder_hyperparameter_search,
)

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


# SeqSigNet with Attention Network

In [7]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = True

In [8]:
num_epochs = 100
dimensions = [15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (10, 3, 5)]
num_layers = [1]
ffn_hidden_dim_sizes = [[32,32], [128,128], [512,512]]
dropout_rates = [0.1]
learning_rates = [5e-4, 3e-4, 1e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 3

In [9]:
kwargs = {
    "num_epochs": num_epochs,
    "df": df_rumours,
    "id_column": "timeline_id",
    "label_column": "label",
    "embeddings": sbert_embeddings,
    "y_data": y_data,
    "output_dim": output_dim,
    "dimensions": dimensions,
    "log_signature": True,
    "pooling": "signature",
    "transformer_encoder_layers": 2,
    "swmhau_parameters": swmhau_parameters,
    "num_layers": num_layers,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "split_ids": split_ids,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# history_length=11

In [10]:
shift = 3
window_size = 5
n = 3

## Random Projections

In [11]:
(
    seqsignet_attention_encoder_umap_11,
    best_seqsignet_attention_encoder_umap_11,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_encoder_umap_focal_2_3_5_3_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_encoder_umap_focal_2_3_5_3_kfold_best_model.csv


In [12]:
seqsignet_attention_encoder_umap_11.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_1817425/2250631359.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seqsignet_attention_encoder_umap_11.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         10              3         5         1          (32, 32)       0.1          0.0001         0.690525   
                                                                                      0.0003         0.689966   
                                                                                      0.0005         0.697856   
                                                          (128, 128)     0.1          0.0001         0.689407   
                                                                                      0.0003         0.683691   
                                                                                      0.0005         0.688537   
                                                          (512, 512)     0.1          0.0001         0.697608   
                                                                                      0.0003         0.689904   
                                                                                      0.0005         0.699534   
           12              3         10        1          (32, 32)       0.1          0.0001         0.688910   
                                                                                      0.0003         0.683628   
                                                                                      0.0005         0.695371   
                                                          (128, 128)     0.1          0.0001         0.688599   
                                                                                      0.0003         0.680149   
                                                                                      0.0005         0.691333   
                                                          (512, 512)     0.1          0.0001         0.680957   
                                                                                      0.0003         0.689966   
                                                                                      0.0005         0.697235   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         10              3         5         1          (32, 32)       0.1          0.0001         0.659447   
                                                                                      0.0003         0.658843   
                                                                                      0.0005         0.667311   
                                                          (128, 128)     0.1          0.0001         0.665119   
                                                                                      0.0003         0.661897   
                                                                                      0.0005         0.664256   
                                                          (512, 512)     0.1          0.0001         0.666834   
                                                                                      0.0003         0.662866   
                                                                                      0.0005         0.667152   
           12              3         10        1          (32, 32)       0.1          0.0001         0.656936   
                                                                                      0.0003         0.657657   
                                                                                      0.0005         0.654197   
                                                          (128, 128)     0.1          0.0001         0.663851   
                                                                                      0.0003         0.663115   
                                

In [13]:
best_seqsignet_attention_encoder_umap_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.662815,0.648297,"[0.7197521301316809, 0.5768421052631579]",0.648562,"[0.7829457364341085, 0.5141784820683903]",0.661449,"[0.6659977064220184, 0.6568993074054342]",None,0.747136,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.699907,0.681218,"[0.7584033613445377, 0.604033448106247]",0.678321,"[0.7956549118387909, 0.5609867519415258]",0.689360,"[0.7244839449541285, 0.6542354821523708]",None,0.734196,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.680149,0.658494,"[0.7444907683144729, 0.5724962630792227]",0.656071,"[0.7744733581164808, 0.5376696303228825]",0.664445,"[0.716743119266055, 0.6121470431539691]",None,0.724222,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [14]:
best_seqsignet_attention_encoder_umap_11[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(512, 512)",0.1,0.0001
0,15,12,3,10,1,"(512, 512)",0.1,0.0001
0,15,12,3,10,1,"(512, 512)",0.1,0.0001


in path: 0.658

in input: 0.6430702845314854

both: 0.6596275491973381

In [15]:
best_seqsignet_attention_encoder_umap_11["f1"].mean()

0.66266967937322

In [16]:
best_seqsignet_attention_encoder_umap_11["precision"].mean()

0.6609848117870298

In [17]:
best_seqsignet_attention_encoder_umap_11["recall"].mean()

0.6717511005589959

In [18]:
np.stack(best_seqsignet_attention_encoder_umap_11["f1_scores"]).mean(axis=0)

array([0.74088209, 0.58445727])

In [19]:
np.stack(best_seqsignet_attention_encoder_umap_11["precision_scores"]).mean(axis=0)

array([0.784358  , 0.53761162])

In [20]:
np.stack(best_seqsignet_attention_encoder_umap_11["recall_scores"]).mean(axis=0)

array([0.70240826, 0.64109394])

# history_length=20

In [21]:
shift = 3
window_size = 5
n = 6

## Random Projections

In [ ]:
(
    seqsignet_attention_encoder_umap_20,
    best_seqsignet_attention_encoder_umap_20,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
seqsignet_attention_encoder_umap_20.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_seqsignet_attention_encoder_umap_20

In [ ]:
best_seqsignet_attention_encoder_umap_20[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_seqsignet_attention_encoder_umap_20["f1"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_20["precision"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_20["recall"].mean()

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_20["recall_scores"]).mean(axis=0)

# history_length=35

In [ ]:
shift = 3
window_size = 5
n = 11

## Random Projections

In [ ]:
(
    seqsignet_attention_encoder_umap_35,
    best_seqsignet_attention_encoder_umap_35,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

In [ ]:
seqsignet_attention_encoder_umap_35.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_seqsignet_attention_encoder_umap_35

In [ ]:
best_seqsignet_attention_encoder_umap_35[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_seqsignet_attention_encoder_umap_35["f1"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_35["precision"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_35["recall"].mean()

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_35["recall_scores"]).mean(axis=0)

# history_length=80

In [ ]:
shift = 3
window_size = 5
n = 26

## UMAP

## Random Projections

In [ ]:
(
    seqsignet_attention_encoder_umap_80,
    best_seqsignet_attention_encoder_umap_80,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

In [ ]:
seqsignet_attention_encoder_umap_80.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_seqsignet_attention_encoder_umap_80

In [ ]:
best_seqsignet_attention_encoder_umap_80[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_seqsignet_attention_encoder_umap_80["f1"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_80["precision"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_80["recall"].mean()

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_80["recall_scores"]).mean(axis=0)